# Play Your Dogs Right validation

Import the required libraries.

In [ ]:
import random
from itertools import product
from pprint import pprint as pp

Generate potential outcomes.

In [ ]:
traps = list(range(1, 7))
potential_outcomes = list(product(traps, traps, traps, traps, traps, traps))

assert len(potential_outcomes) == 6**6
assert potential_outcomes[0] == (1, 1, 1, 1, 1, 1)
assert potential_outcomes[-1] == (6, 6, 6, 6, 6, 6)

Determines which group the player's bet belongs to.

In [ ]:
def get_longest_contiguous_sequence(selection):
    current_sequence_length = 0
    longest_sequence_length = 0

    for i in range(len(selection)):
        if i == 0 or selection[i] == selection[i - 1]:
            current_sequence_length += 1
        else:
            if current_sequence_length > longest_sequence_length:
                longest_sequence_length = current_sequence_length

            current_sequence_length = 1

    return max(longest_sequence_length, current_sequence_length)

In [ ]:
def get_bet_group(selection):
    return min(get_longest_contiguous_sequence(selection), 5)

Calculate length of run.

In [ ]:
def get_run_length(selection, outcome):
    previous_trap = 3.5
    result = 0

    for i in range(len(selection)):
        if (selection[i] == "L" and outcome[i] < previous_trap) or (
            selection[i] == "H" and outcome[i] > previous_trap
        ):
            result += 1
        elif i > 0:
            break

        previous_trap = outcome[i]

    return result

Calculate dividend.

In [ ]:
def get_dividend(bet_group, run_length):
    match bet_group, run_length:
        case 1, 6:
            return 8
        case 1, 5:
            return 5
        case 1, 4:
            return 3
        case 1, 3:
            return 2
        case 2, 6:
            return 13
        case 2, 5:
            return 5
        case 2, 4:
            return 3
        case 2, 3:
            return 2
        case 3, 6:
            return 40
        case 3, 5:
            return 9
        case 3, 4:
            return 3
        case 3, 3:
            return 2
        case 4, 6:
            return 400
        case 4, 5:
            return 30
        case 4, 4:
            return 5
        case 4, 3:
            return 2
        case 5, 6:
            return 10_000
        case 5, 5:
            return 250
        case 5, 4:
            return 20
        case 5, 3:
            return 3
        case _:
            return 0

Calculate RTP.

In [ ]:
def calculate_rtp(selection):
    bet_group = get_bet_group(selection)

    total_dividends = 0

    for outcome in potential_outcomes:
        run_length = get_run_length(selection, outcome)
        total_dividends += get_dividend(bet_group, run_length)

    return total_dividends / len(potential_outcomes)

Create all possible selections.

In [ ]:
options = ["L", "H"]

selections = [
    "".join(selection)
    for selection in product(options, options, options, options, options, options)
]

assert len(selections) == 64
assert selections[0] == "LLLLLL"
assert selections[-1] == "HHHHHH"

Calculate RTPs for all selections, and sort by descending RTP.

In [ ]:
rtps = sorted(
    [(selection, calculate_rtp(selection) * 100) for selection in selections],
    key=lambda x: x[1],
    reverse=True,
)

Display all RTPs (for all selections).

In [ ]:
for selection, rtp in rtps:
    print(f"RTP for selection {selection} = {rtp:.1f}")

Perform consistency checks with Lisa's numbers

In [ ]:
def get_bet_group_selections(selections, bet_group):
    return [
        selection for selection in selections if get_bet_group(selection) == bet_group
    ]

In [ ]:
group_1_rtps = sorted(
    [
        (selection, calculate_rtp(selection) * 100)
        for selection in get_bet_group_selections(selections, 1)
    ],
    key=lambda x: x[1],
    reverse=True,
)

In [ ]:
assert round(group_1_rtps[0][1], 2) == 85.16

In [ ]:
group_2_rtps = sorted(
    [
        (selection, calculate_rtp(selection) * 100)
        for selection in get_bet_group_selections(selections, 2)
    ],
    key=lambda x: x[1],
    reverse=True,
)

In [ ]:
assert group_2_rtps[0][0] == "LHLHLL"
assert round(group_2_rtps[0][1], 2) == 85.25

In [ ]:
group_3_rtps = sorted(
    [
        (selection, calculate_rtp(selection) * 100)
        for selection in get_bet_group_selections(selections, 3)
    ],
    key=lambda x: x[1],
    reverse=True,
)

In [ ]:
assert group_3_rtps[0][0] == "LHLHHH"
assert group_3_rtps[-2][0] == "LLLHHH"
assert round(group_3_rtps[-2][1], 2) == 18.35
assert group_3_rtps[-1][0] == "HHHLLL"